# **PYNQ Juliabrot Fractal Factory v1.0 [for Jupyter Lab on Ultra96 and PYNQ Z1/Z2]**
Created by: Fred Kellerman, [Github Repo](https://github.com/FredKellerman/pynq-juliabrot)

![mandelbrot](./large-images/f3.png) ![mandelbrot](./large-images/f4.png) ![mandelbrot](./large-images/f5.png)

### **Important:** ipycanvas 0.4.7 Jupyter Lab extension widget must be installed before using this notebook.
See Appendix for how-to: [here](#installipycanvas)

# Infinite graphical math from a simple complex number iterative equation

$z \rightarrow z^2 + c$

See Wikipedia: [Julia Set](https://en.wikipedia.org/wiki/Julia_set) and [Mandelbrot Set](https://en.wikipedia.org/wiki/Mandelbrot_set)

### Setup

In [ ]:
from juliabrot import JuliabrotGrid, JuliabrotTile, JuliabrotGridSettings
import juliabrot_ui as jui
import juliabrot_coloring as jcolor
import numpy as np
# If you see 'Javascript Error: require is not defined' below, it is a PYNQ thing and you can ignore it, it is benign

### Enter intial fractal settings from code (example)

In [ ]:
# Setup the global grid settings, these will render the overview of the classic Mandelbrot set
js = JuliabrotGridSettings()
js.sizeX = 1920/4             # Pixel width of generated image (may be padded)
js.sizeY = 1080/4             # Pixel height of generated image
js.max_iterations = 255       # Max iterations
js.mandelbrot_mode = True     # Set False to generate Julia set
js.ulX = np.longdouble(-2.5)  # Upper-left X
js.ulY = np.longdouble(1.5)   # Upper-left Y
js.lrX = np.longdouble(2.5)   # Lower-right X
js.lrY = np.longdouble(-2.5)  # Lower-right Y
js.cX  = np.longdouble(0.388) # Julia set X point
js.cY  = np.longdouble(0.312) # Julia set Y point

# Leave empty to auto-select or choose to set to:
#  64 - 6x kernels @ 64bits, 95 - 4x kernels @ 95 bits, 160 - 1x kernels @ 160bits (all @ 300MHz)
#  64 is the fastest, 95 in the middle for speed, 160 the slowest but highest precision
# PYNQ Z1-Z2 boards have 1 overlay for 3x kernels @ 64 bits ~ 125MHz
js.kernel_mode = 64

# Create a grid object, new auto generated tile size defaults to same size as full grid,
jg = JuliabrotGrid(js)

# Initialize the GUI, render it later
jui.init_ui(jg)

### Or load in a json preset file to setup the intial conditions (example)

In [ ]:
# Uncomment to load in json saved setting file, clicking the save button in the GUI creates json files.
#  From code you can evoke the save_json method (see juliabrot.py)
#jui.init_ui('./catalog/juliabrot_0x7a5c2bf9_08_09_2020-02_14_51.json')

## Interactive live preview

Use mouse to select area to zoom into.  Bump buttons move number of bump pixels, up/down/left/right.  Use sliders and pull-downs to set interations and coloring options.  Save button will create json file in notebook's catalog folder.

You may double click in the same spot on the display to set the Julia coordinates.  Setting Zoom to value 0.0 and clicking Zoom button will reset to original coordinates but will not reset any other settings.

In [ ]:
jui.draw_roaming_ui()  # Execute to draw interactive GUI below
# Click once over fractal to start selection of upper-left, click again to end lower-right selection

## How-to generate larger image

Once you have a created a likeable preview you can easily replicate it and create a larger version.  Here's how:

In [ ]:
# Re-use GUI preview settings
import copy
big_grid = copy.deepcopy(jui.jgrid)

In [ ]:
# Change some of the preview grid settings, for example size. Fractal will generate below cell
#  Max dimensions are 16K x 16K, beware PS memory is limited and large images may have issues when coloring
#  Note: cannot override kernel_mode unless overlay is reloaded!
big_grid.set_size(1920,1080)
big_grid.settings.max_iterations = int(jui.iter_slider.value)
big_tile = jui.juliabrot.compute(big_grid.tile_list[0], True)

## Choose a method to color the larger fractal

You may choose some various coloring methods (or create your own).  To match the preview, uncomment the appropriate method and run (just one):

Note: large images may exhuast the CPU memory, I've been able to color around 4096x2160 on U96, YMMV.  Coloring is done in Python software (it can take a while).  Keep in mind Z1/Z2 has less memory than U96.

In [ ]:
# No color just Black and White
#rgb = big_tile.data.iterations

In [ ]:
#rgb = jcolor.rgb_iter_max(big_tile, h=jui.hue_slider.value, s=jui.sat_slider.value, v=jui.val_slider.value, modulo=jui.modulo_slider.value, in_colors=[jui.picker1.value])

In [ ]:
#rgb = jcolor.color_classic(big_tile, h= jui.hue_slider.value, s=jui.sat_slider.value, v=jui.val_slider.value, modulo=jui.modulo_slider.value, in_colors=[jui.picker1.value])

In [ ]:
#rgb = jcolor.color_log(big_tile, h= jui.hue_slider.value, s=jui.sat_slider.value, v=jui.val_slider.value, modulo=jui.modulo_slider.value, in_colors=[jui.picker1.value])

In [ ]:
rgb = jcolor.color_rainbow(big_tile, h=jui.hue_slider.value, s=jui.sat_slider.value, v=jui.val_slider.value, modulo=jui.modulo_slider.value, in_colors=[jui.picker1.value])

In [ ]:
#rgb = jcolor.color_rainbow2(big_tile, h=jui.hue_slider.value, s=jui.sat_slider.value, v=jui.val_slider.value, modulo=jui.modulo_slider.value, in_colors=[jui.picker1.value])

## After coloring the iteration data display it
This may have troubles for 4K or larger images especially when saving canvas, you can try skipping this and just save to PNG then view the file

In [ ]:
# Create the canvas, push the data to it
canvas_full = jui.Canvas(width=big_tile.sizeX, height=big_tile.sizeY)
canvas_full.sync_image_data = True
with jui.hold_canvas(canvas_full) :
    canvas_full.put_image_data(rgb, 0, 0)
    canvas_full.fill_style="#ffffff"  # Set text color
    canvas_full.fill_text('PYNQ Juliabrot', 10, big_grid.settings.sizeY - 20) # Optionally write some text on your image

In [ ]:
# Display the image
canvas_full

## Optionally save the large fractal as a PNG file

Save the above canvas as an 8-bit PNG file, canvas.to_file() can have issues for large 4K sized images, use the 2nd method if it does

In [ ]:
canvas_full.to_file('./user-images/fractal.png')

In [ ]:
# Uncomment alternative and slower method to save to png (but it won't have the text)
#from skimage.io import imsave
#imsave('./user-images/fractal.png', rgb.astype(np.uint8)) # rgb is 64-bit float and scaled for 8-bit color

## If larger images aren't cooperating for display or writing to a PNG file or you want to color faster
Consider writing the Numpy array to a file and re-import into a PC Python environment and convert into image there

In [ ]:
# Something like this:
#with open('fractal-rgb.npy', 'wb') as f:
#   np.save(f, rgb)
#with open('fractal-iter.npy', 'wb') as f:
#   np.save(f, big_tile.data.iterations)  # Can save iterations also!
# Read it back in like this:
#with open('fractal-rgb.npy', 'rb') as f:
#   fractalRgb = np.load(f)

## A few more examples

![Mandelbrot](./large-images/title_fractal.png) ![Mandelbrot](./large-images/f2.png) ![Mandelbrot](./large-images/f6.png) ![Mandelbrot](./large-images/f7.png)

<a id="installipycanvas"></a>

## Appendix: how to install ipycanvas 0.4.7

Open a console and enter the follwing commands, the last will take a while to execute:
```bash
sudo pip3 install ipycanvas==0.4.7
sudo jupyter labextension install @jupyter-widgets/jupyterlab-manager ipycanvas@0.4.7 --no-build
sudo jupyter lab build --minimize=False
sudo jupyter labextension install ipycanvas@0.4.7 --minimize=False
```
After completion either restart the jupyter lab service (or simply reboot).  Then restart this Notebook.

## Origins

This notebook was inspired by a dog eared copy of the book: "Fractals, Chaos, Power Laws: Minutes from an Infinite Paradise", written by the late Dr. Manfred Schroeder, published and Copyright 1991 by W.H. Freeman

![Published by Dover](./large-images/fractals_chaos_schroeder.jpg)